In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from os import path
try:
    import urllib
except ImportError:
    from urlparse import urlparse
import pandas
# import sys
# !{sys.executable} -m pip install lxml
import lxml
import html5lib

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:10.0.1) Gecko/20100101 Firefox/10.0.1'}

r = requests.get('https://www.statista.com/statistics/227249/greatest-gap-between-rich-and-poor-by-us-state/', headers=headers)

root = BeautifulSoup(r.text, 'html.parser')
table = root.find("table").prettify()

income_list = pandas.read_html(str(table), converters={"tbody": str})
income_table = pandas.DataFrame(income_list[0])
income_table.columns = ['States', 'Gini Coefficient']

In [ ]:
crime = requests.get('https://www.statista.com/statistics/200445/reported-violent-crime-rate-in-the-us-states/', headers=headers)

crime_soup = BeautifulSoup(crime.text, 'html.parser')

# View the content and find the appropriate table
crime_table = crime_soup.find("table").prettify()

# Save the aforementioned table as a variable
crime_list = pandas.read_html(str(crime_table), converters={"tbody": str})
crime_list = crime_list[0]
crime_list.columns = ['States', 'Crime Rate']

data = crime_list.merge(income_table, how = "inner", left_on = "States", right_on = "States")

data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame
import seaborn as sns

# sns.lineplot(data=data)

# fig, ax1 = plt.subplots()
# ax2 = ax1.twinx()  
# sns.regplot(x='States', y='Gini Coefficient', data=data, order=2, ax=ax1)
# sns.regplot(x='States', y='Crime Rate', data=data, order=2, ax=ax2)
# ax2.legend(handles=[a.lines[0] for a in [ax1,ax2]], 
#            labels=["f", "g"])
# plt.show()


fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('States')
ax1.set_ylabel('Gini Coefficient', color=color)
ax1.plot(data['States'], data['Gini Coefficient'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Crime Rate', color=color)  # we already handled the x-label with ax1
ax2.plot(data['States'], data['Crime Rate'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

# data.plot(x="States", y=["Gini Coefficient", "Crime Rate"], kind="bar")

# plt.show()